In [1]:
import numpy as np
from scipy.stats import norm
from scipy.integrate import quad

def gaussian_overlap(mu1, sigma1, mu2, sigma2):
    # Define the PDFs of the two Gaussian distributions
    def pdf1(x):
        return norm.pdf(x, mu1, sigma1)
    
    def pdf2(x):
        return norm.pdf(x, mu2, sigma2)
    
    # Define the integrand as the minimum of the two PDFs
    def integrand(x):
        return np.minimum(pdf1(x), pdf2(x))
    
    # Calculate the overlap by integrating the minimum PDF
    overlap, _ = quad(integrand, -np.inf, np.inf)
    
    # Convert to percentage
    overlap_percentage = overlap * 100
    
    return overlap_percentage

# Example usage:
mu1, sigma1 = 0, 1  # Mean and standard deviation for the first Gaussian
mu2, sigma2 = 0, 1  # Mean and standard deviation for the second Gaussian

overlap_percentage = gaussian_overlap(mu1, sigma1, mu2, sigma2)
print(f"The percentage of overlap between the two Gaussian distributions is: {overlap_percentage:.2f}%")


The percentage of overlap between the two Gaussian distributions is: 100.00%
